In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import pandas as pd

from fredlib import preprocessText, getFredGraph
import time


In [2]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [4]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [5]:
def from_text_to_kb(text, span_length=128, verbose=False):
    
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [6]:
text = """
Hillary Clinton is in the political crosshairs as the author of a new book alleges improper financial ties between her public and personal life. At issue in conservative author Peter Schweizer’s forthcoming book Clinton Cash are donations from foreign governments to the Clinton Foundation during the four years she served as secretary of state. George Stephanopoulos used an interview with Schweizer on ABC This Week to point out what other nonpartisan journalists have found: There is no ""smoking gun"" showing that donations to the foundation influenced her foreign policy decisions. Still, former Republican House Speaker Newt Gingrich says the donations are ""clearly illegal"" under federal law. In his view, a donation by a foreign government to the Clinton Foundation while Clinton was secretary of state is the same as money sent directly to her, he said, even though she did not join the foundation’s board until she left her post. ""The Constitution of the United States says you cannot take money from foreign governments without explicit permission of the Congress. They wrote that in there because they knew the danger of corrupting our system by foreign money is enormous,"" Gingrich said. ""You had a sitting secretary of state whose husband radically increased his speech fees, you have a whole series of dots on the wall now where people gave millions of dollars — oh, by the way, they happen to get taken care of by the State Department."" He continued, ""My point is they took money from foreign governments while she was secretary of State. That is clearly illegal."" PunditFact wanted to know if a criminal case against Clinton is that open and shut. Is what happened ""clearly illegal""? A spokesman for the Clinton Foundation certainly disagreed, calling Gingrich’s accusation ""a baseless leap"" because Clinton was not part of her husband’s foundation while serving as a senator or secretary of state. We did not hear from Gingrich by our deadline. Foundation basics Former President Clinton started the William J. Clinton Foundation in 2001, the year after Hillary Clinton won her first term as a New York senator. The foundation works with non-governmental organizations, the private sector and governments around the world on health, anti-poverty, HIV/AIDS and climate change initiatives. Spokesman Craig Minassian said it’s reasonable for the foundation to accept money from foreign governments because of the global scope of its programs, and the donations are usually in the form of tailored grants for specific missions. Hillary Clinton was not part of her husband’s foundation while she was a senator or secretary of state. Her appointment to the latter post required Senate confirmation and came with an agreement between the White House and Clinton Foundation that the foundation would be more transparent about its donors. According to the 2008 memorandum of understanding, the foundation would release information behind new donations and could continue to collect donations from countries with which it had existing relationships or running grant programs. If countries with existing contributions significantly stepped up their contributions, or if a new foreign government wanted to donate, the State Department would have to approve. Clinton took an active role in fundraising when she left the State Department and the foundation became the Bill, Hillary & Chelsea Clinton Foundation in 2013. But she left the board when she announced her run for the presidency in April 2015. The Emoluments Clause So how does Gingrich come up with the claim that Clinton Foundation donations are ""clearly illegal"" and unconstitutional? The answer is something known as the Emoluments Clause. A few conservative websites have made similar arguments in recent days, including the Federalist blog. The Emoluments Clause, found in Article 1, Section 9 of the Constitution, reads in part: ""No Title of Nobility shall be granted by the United States: And no Person holding any Office of Profit or Trust under them, shall, without the Consent of the Congress, accept of any present, Emolument, Office, or Title, of any kind whatever, from any King, Prince, or foreign State."" The framers came up with this clause to prevent the government and leaders from granting or receiving titles of nobility and to keep leaders free of external influence. (An emolument, per Merriam-Webster Dictionary, is ""the returns arising from office or employment usually in the form of compensation or perquisites."") Lest you think the law is no longer relevant, the Pentagon ethics office in 2013 warned employees the ""little known provision"" applies to all federal employees and military retirees. There’s no mention of spouses in the memo. J. Peter Pham, director of the Atlantic Council’s Africa Center, said interpretation of the clause has evolved since its adoption at the Constitutional Convention, when the primary concern was about overseas diplomats not seeking gifts from foreign powers they were dealing with. The Defense Department memo, in his view, goes beyond what the framers envisioned for the part of the memo dealing with gifts. ""I think that, aside from the unambiguous parts, the burden would be on those invoking the clause to show actual causality that would be in violation of the clause,"" Pham said. Expert discussion We asked seven different constitutional law experts on whether the Clinton Foundation foreign donations were ""clearly illegal"" and a violation of the Emoluments Clause. We did not reach a consensus with their responses, though a majority thought the layers of separation between the foundation and Hillary Clinton work against Gingrich. The American system often distinguishes between public officers and private foundations, ""even if real life tends to blur some of those distinctions,"" said American University law professor Steve Vladeck. Vladeck added that the Emoluments Clause has never been enforced. ""I very much doubt that the first case in its history would be because a foreign government made charitable donations to a private foundation controlled by a government employee’s relative,"" he said. ""Gingrich may think that giving money to the Clinton Foundation and giving money to then-Secretary Clinton are the same thing. Unfortunately for him, for purposes of federal regulations, statutes, and the Constitution, they’re formally — and, thus, legally — distinct."" Robert Delahunty, a University of St. Thomas constitutional law professor who worked in the Justice Department’s Office of Legal Counsel from 1989 to 2003, also called Gingrich’s link between Clinton and the foreign governments’ gifts to the Clinton Foundation as ""implausible, and in any case I don’t think we have the facts to support it."" ""The truth is that we establish corporate bodies like the Clinton Foundation because the law endows these entities with a separate and distinct legal personhood,"" Delahunty said. John Harrison, University of Virginia law professor and former deputy assistant attorney general in the Office of Legal Counsel from 1990 to 1993, pointed to the Foreign Gifts Act, 5 U.S.C. 7432, which sets rules for how the Emoluments Clause should work in practice. The statute spells out the minimal value for acceptable gifts, and says it applies to spouses of the individuals covered, but ""it doesn’t say anything about receipt of foreign gifts by other entities such as the Clinton Foundation."" ""I don’t know whether there’s any other provision of federal law that would treat a foreign gift to the foundation as having made to either of the Clintons personally,"" Harrison said, who added that agencies have their own supplemental rules for this section, and he did not know if the State Department addressed this. Other experts on the libertarian side of the scale thought Gingrich was more right in his assertion. Clinton violates the clause because of its intentionally broad phrasing about gifts of ""any kind whatever,"" which would cover indirect gifts via the foundation, said Dave Kopel, a constitutional law professor at Denver University and research director at the libertarian Independence Institute. Kopel also brought up bribery statutes, which would require that a gift had some influence in Clinton’s decision while secretary of state. Delahunty thought Kopel’s reasoning would have ""strange consequences,"" such as whether a state-owned airline flying Bill Clinton to a conference of former heads of state counted as a gift to Hillary Clinton. Our ruling Gingrich said the Clinton Foundation ""took money from from foreign governments while (Hillary Clinton) was secretary of state. It is clearly illegal. … The Constitution says you can’t take this stuff."" A clause in the Constitution does prohibit U.S. officials such as former Secretary of State Hillary Clinton from receiving gifts, or emoluments, from foreign governments. But the gifts in this case were donations from foreign governments that went to the Clinton Foundation, not Hillary Clinton. She was not part of the foundation her husband founded while she was secretary of state. Does that violate the Constitution? Some libertarian-minded constitutional law experts say it very well could. Others are skeptical. What’s clear is there is room for ambiguity, and the donations are anything but ""clearly illegal."" The reality is this a hazy part of U.S. constitutional law.
"""

kb = from_text_to_kb(text, verbose=True)
kb.print()

Token indices sequence length is longer than the specified maximum sequence length for this model (1834 > 1024). Running this sequence through the model will result in indexing errors


Input has 1834 tokens
Input has 15 spans
Span boundaries are [[0, 128], [121, 249], [242, 370], [363, 491], [484, 612], [605, 733], [726, 854], [847, 975], [968, 1096], [1089, 1217], [1210, 1338], [1331, 1459], [1452, 1580], [1573, 1701], [1694, 1822]]
Relations:
  {'head': 'Hillary Clinton', 'type': 'position held', 'tail': 'secretary of state', 'meta': {'spans': [[0, 128], [363, 491], [484, 612]]}}
  {'head': 'Clinton Cash', 'type': 'author', 'tail': 'Peter Schweizer', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Newt Gingrich', 'type': 'position held', 'tail': 'House Speaker', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Newt Gingrich', 'type': 'member of political party', 'tail': 'Republican', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Constitution of the United States', 'type': 'main subject', 'tail': 'Congress', 'meta': {'spans': [[121, 249]]}}
  {'head': 'Clinton Foundation', 'type': 'named after', 'tail': 'Constitution of the United States', 'meta': {'spans': [[121, 249]]}}
  {'hea

In [7]:
len(kb.relations)

45

In [40]:
train = pd.read_csv('train.tsv',sep='\t')
test = pd.read_csv('test.tsv',sep='\t')
dev = pd.read_csv('dev.tsv', sep='\t')

train = train.dropna(subset=['claim', 'main_text'])
test = test.dropna(subset=['claim', 'main_text'])
dev = dev.dropna(subset=['claim', 'main_text'])

In [9]:
train.shape

(9806, 9)

In [11]:
train['claim'][:10].apply(lambda claim: from_text_to_kb(claim).relations)

0    [{'head': 'Hillary Clinton', 'type': 'position...
1    [{'head': 'False-Positives', 'type': 'facet of...
2    [{'head': 'SBRT', 'type': 'medical condition t...
3    [{'head': 'Ovarian Cancer', 'type': 'has part'...
4    [{'head': 'emergency', 'type': 'subclass of', ...
5    [{'head': 'coronavirus', 'type': 'has effect',...
6    [{'head': 'new study', 'type': 'main subject',...
7    [{'head': 'healthcare delivery reforms', 'type...
8    [{'head': 'J&J', 'type': 'product or material ...
9    [{'head': 'drug', 'type': 'part of', 'tail': '...
Name: claim, dtype: object

In [ ]:
train['claim_kg'] = train['claim'].apply(lambda claim: from_text_to_kb(claim).relations)

In [ ]:
train.to_pickle("train.pkl")

In [12]:
train['main_text'][:10].apply(lambda main_text: from_text_to_kb(main_text).relations)

0    [{'head': 'Clinton Cash', 'type': 'author', 't...
1    [{'head': 'recall mammography', 'type': 'subcl...
2    [{'head': 'Robert Meier', 'type': 'occupation'...
3    [{'head': 'Provenge', 'type': 'instance of', '...
4    [{'head': 'anesthesiologist', 'type': 'part of...
5    [{'head': 'health minister', 'type': 'officeho...
6    [{'head': 'hospitalization', 'type': 'differen...
7    [{'head': 'healthcare reform law', 'type': 'au...
8    [{'head': 'Alameda Superior Court', 'type': 'l...
9    [{'head': 'Zocor', 'type': 'owned by', 'tail':...
Name: main_text, dtype: object

In [71]:
test.shape

(1235, 10)

In [72]:
test['claim_kg'] = test['claim'].apply(lambda claim: from_text_to_kb(claim).relations)

In [74]:
test.to_pickle("test.pkl")

In [77]:
test['claim_kg'].describe()

count                                                  1235
unique                                                 1231
top       [{'head': 'HIV', 'type': 'has effect', 'tail':...
freq                                                      3
Name: claim_kg, dtype: object

In [84]:
test['main_text_kg'] = test['main_text'].apply(lambda main_text: from_text_to_kb(main_text).relations)

In [87]:
test['main_text_kg'].describe()

count                                                  1235
unique                                                 1231
top       [{'head': 'Coronavirus pneumonia', 'type': 'su...
freq                                                      2
Name: main_text_kg, dtype: object

In [86]:
test.to_pickle("test.pkl")

In [88]:
dev['claim_kg'] = dev['claim'].apply(lambda claim: from_text_to_kb(claim).relations)

In [89]:
dev.to_pickle("dev.pkl")

In [90]:
dev['main_text_kg'] = dev['main_text'].apply(lambda main_text: from_text_to_kb(main_text).relations)

In [91]:
dev.to_pickle("dev.pkl")

In [41]:
dev['main_text'][:1].apply(lambda claim: from_text_to_kb(claim).relations)[0]

[{'head': 'Confederate flag',
  'type': 'instance of',
  'tail': 'racist',
  'meta': {'spans': [[0, 128]]}},
 {'head': 'FierceFemtivist',
  'type': 'publication date',
  'tail': '8 November 2015',
  'meta': {'spans': [[0, 128]]}},
 {'head': 'FierceFemtivist',
  'type': 'point in time',
  'tail': '8 November 2015',
  'meta': {'spans': [[0, 128]]}},
 {'head': 'Confederate flag',
  'type': 'applies to jurisdiction',
  'tail': 'Confederate',
  'meta': {'spans': [[100, 228]]}},
 {'head': 'Confederate',
  'type': 'owner of',
  'tail': 'Confederate flag',
  'meta': {'spans': [[100, 228]]}},
 {'head': 'stock redneck',
  'type': 'subclass of',
  'tail': 'racism',
  'meta': {'spans': [[200, 328]]}},
 {'head': 'Confederate-flag',
  'type': 'political ideology',
  'tail': 'redneck',
  'meta': {'spans': [[200, 328]]}},
 {'head': 'redneck',
  'type': 'subclass of',
  'tail': 'racism',
  'meta': {'spans': [[200, 328]]}},
 {'head': 'controversial tweets',
  'type': 'author',
  'tail': '@FierceFemtivis

In [42]:
dev[:1]

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism"


In [43]:
dev['explanation'][0]

'Fellow Twitter users suggested @FierceFemtivist indicated she was from Louisville, Kentucky, prior to deleting her account. We were unable to locate any news reports of unusual infant deaths on 8 November 2015 in that area, nor were we able to confirm that the purported incident took place in Louisville (or at all):'

In [44]:
train['label'].value_counts()

label
true        5078
false       3001
mixture     1434
unproven     291
snopes         1
Name: count, dtype: int64

In [53]:
train[train['label']=='mixture'][:1]['claim'][1]

'Annual Mammograms May Have More False-Positives'

In [52]:
train[train['label']=='mixture'][:1]['explanation'][1]

'This article reports on the results of a study of nearly 170,000 women who had screening mammograms beginning between age 40-59. The study found that over ten years of screening mammograms, over half of the women will experience a false-positive recall for additional mammography. In addition, 7%-9% of the women will have a biopsy for a suspicious lump which is not cancerous. Both of those percentages decrease if the woman is screened every other year rather than every year. Even with biennial mammography, 41% of women will experience a\xa0recall over 10 years of mammography. The study’s Principal Investigator emphasized that “in most cases, a recall doesn’t mean you have cancer.”\xa0 She hoped this knowledge would reduce the anxiety of women who are recalled. The story never explained the size of the decrease in the number of false positives between annual (61.3%) and biennial screening (41.6%). Our first two reviewers were a researcher who specializes in health decisions and a breast

In [54]:
train[train['label']=='mixture'][:1]['main_text'][1]

'While the financial costs of screening mammography & false-positive recalls & biopsies were not included in the study, readers would benefit from being reminded that recall mammography & biopsies increase patient financial costs. This article leads with valuable information for all women over age 40 by stating “Getting screening mammograms every two years instead of annually reduces the chance of a false alarm, a new study shows.”\xa0 Unfortunately the writer doesn’t quantify or elaborate adequately on that reduction. Instead, the writer later focuses on how women undergoing screening mammography shouldn’t be anxious if they experience a recall because over half of women who have screening mammograms for 10 years will be recalled. Readers would have been better served if the writer had emphasized the significant reduction in both recall mammograms & false-positive biopsies in women who are screened every two years rather than annually. Part of the continuing controversy\xa0over screen

In [62]:
claim = "The earth is flat, like a disk"
evidence = "NASA studies show that the earth is round."

In [79]:
claim_relations = from_text_to_kb(claim).relations

In [81]:
for rdf_triple in claim_relations:
    print(f"({rdf_triple['head']}, {rdf_triple['type']}, {rdf_triple['tail']})")

(earth is flat, said to be the same as, disk)
(disk, said to be the same as, earth is flat)
(flat, facet of, earth)
(flat, said to be the same as, disk)
(disk, said to be the same as, flat)


In [82]:
evidence_relations = from_text_to_kb(evidence).relations

In [83]:
for rdf_triple in evidence_relations:
    print(f"({rdf_triple['head']}, {rdf_triple['type']}, {rdf_triple['tail']})")

(round, part of, earth)
(earth is round, creator, NASA)
(earth is round, discoverer or inventor, NASA)


In [41]:
dev_df = pd.read_pickle('dev.pkl')
dev_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc..."
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'..."
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t..."


In [42]:
test_df = pd.read_pickle('test.pkl')
test_df.head()

,Unnamed: 0,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg
0,0,33456,A mother revealed to her child in a letter aft...,"November 6, 2011",The one-eyed mother story expounds upon two mo...,David Mikkelson,"In April 2005, we spotted a tearjerker on the ...",,false,Glurge Gallery,"[{'head': 'mother', 'type': 'child', 'tail': '...","[{'head': 'My mom only had one eye', 'type': '..."
1,1,2542,Study says too many Americans still drink too ...,"February 25, 2013","On any given day in the United States, 18 perc...",,That means the great majority of Americans sta...,http://bit.ly/X1NVtW,true,Health News,"[{'head': 'drink too much', 'type': 'facet of'...","[{'head': 'Patricia Guenther', 'type': 'occupa..."
2,2,26678,Viral image Says 80% of novel coronavirus case...,"March 13, 2020",The website Information is Beautiful published...,Paul Specht,"Amid the spread of the novel coronavirus, many...",https://www.facebook.com/informationisbeautifu...,true,"Facebook Fact-checks, Coronavirus, Viral image,","[{'head': 'Mild', 'type': 'subclass of', 'tail...",[{'head': 'Center for Disease Control and Prev...
3,3,40705,An email says that 9-year old Craig Shergold o...,"March 16, 2015",Send greeting or business cards to cancer vict...,Rich Buhler & Staff,Craig Shergold is real and in 1989...,https://www.reddit.com/submit?url=https%3A%2F%...,false,"Inspirational, Pleas","[{'head': 'Craig Shergold', 'type': 'place of ...","[{'head': 'Craig Shergold', 'type': 'date of b..."
4,4,35718,"Employees at a Five Guys restaurant in Daphne,...","July 15, 2020","What's undetermined: As of this writing, Five ...",Dan MacGuill,"In July 2020, amid a new wave of nationwide pr...",,unproven,Law Enforcement,"[{'head': 'Five Guys', 'type': 'location of fo...","[{'head': 'Jubilee Square', 'type': 'located i..."


In [43]:
dev_test_df = pd.concat([dev_df, test_df])

In [44]:
dev_test_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,Unnamed: 0
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",NaN
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...",NaN
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",NaN
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",NaN
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",NaN


In [96]:
dev_test_df.shape

(2452, 12)

In [97]:
dev_test_df.to_pickle('dev_test.pkl')

In [106]:
def print_triples_sizes():
    dev_test_df['claim_kg_size'] = dev_test_df['claim_kg'].apply(lambda triples : len(triples))
    dev_test_df['main_text_kg_size'] = dev_test_df['main_text_kg'].apply(lambda triples : len(triples))

    print(dev_test_df[['claim_kg_size', 'main_text_kg_size']].describe())
    
print_triples_sizes()

       claim_kg_size  main_text_kg_size
count    2452.000000        2452.000000
mean        3.113785          23.469005
std         0.534892          15.511103
min         1.000000           2.000000
25%         3.000000          13.000000
50%         3.000000          21.000000
75%         3.000000          30.000000
max         9.000000         151.000000


In [45]:
def saveGraphFromSentence(sentence, path, key = "fa042f72-771c-3d1b-9dfc-c84eee277cde"):
    return getFredGraph(preprocessText(sentence), key, path)

In [56]:
claims = dev_df['claim'].tolist()
main_texts = dev_df['main_text'].tolist()

print(f"{len(claims)} claims and {len(main_texts)} main_texts to go!")
failed = []
for i in range(len(claims)):
    try:
        saveGraphFromSentence(claims[i], f"./data/claims/{i}_claim.rdf")

        time.sleep(12)

        saveGraphFromSentence(main_texts[i], f"./data/main_texts/{i}_main_text.rdf")

        time.sleep(12)

        if i%100 == 0:
            print(f"{i+1} claims and main_texts are done!")
    except:
        failed.append(i)



1217 claims and 1217 main_texts to go!


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '208.'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '

701 claims and main_texts are done!


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '2.36'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '

1201 claims and main_texts are done!


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#nonNegativeInteger, Converter=<class 'int'>
Traceback (most recent call last):
  File "c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '1.9'
